In [1]:
#! pip install fairlearn
#! pip install lightgbm
#! pip install optuna

In [2]:
from sklearn.metrics import (
    f1_score, 
    confusion_matrix, 
    make_scorer, 
    accuracy_score, 
    recall_score, 
    matthews_corrcoef
)
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

from concurrent.futures import ThreadPoolExecutor
import tqdm as notebook_tqdm
from metrics import (
    equality_opportunity_difference,
    predictive_equality_difference,
    predictive_parity_difference,
    metrics,
    average_absolute_odds_difference
    
)
from fairlearn.metrics import demographic_parity_difference
from sklearn.utils import resample

import numpy as np
import optuna
import dill
import pickle


In [3]:
sensitive_attribute = 'sex'
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1

if sensitive_attribute == 'race':
    mapping = {'White':'white','Black':'black','Asian-Pac-Islander':'others','Amer-Indian-Eskimo':'others','Other':'others'}
    X_raw.loc[:,'race'] = X_raw['race'].map(mapping).astype("category")

A = X_raw[sensitive_attribute]



perc = .5
X_raw, y, A = resample(X_raw, y, A, n_samples=int(perc*X_raw.shape[0]), random_state = 123)

numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)


/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
results = []


In [ ]:
n_trials = 100
num_sim = 4
directions = ['minimize', 'maximize']
metric_scorer_decorated =  metrics(recall_score, 
                                   predictive_parity_difference, 
                                   sensitive_col = sensitive_attribute)
for sim in range(num_sim):
    print(sim)
    file_name = f'recall-ppv-models-motpe-succesivehalving-parallel-{n_trials}trials-{sim+1}sim.pkl'
    def objective(trial):
        (X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
        X_raw, y, A, test_size=0.2, random_state=sim, stratify=y
        )

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        A_train = A_train.reset_index(drop=True)
        A_test = A_test.reset_index(drop=True)

        classifier_name = trial.suggest_categorical("classifier", ["RF", 'GBM','LGBM'])

        if classifier_name == "logit":        
            params = {
                "penalty" : trial.suggest_categorical('logit_penalty', ['l1','l2']),
                "C" : trial.suggest_float('logit_c', 0.001, 10),
                #"solver" : trial.suggest_categorical('logit_solver', ['saga','lbfgs']),
                "max_iter": 2000,
                "solver" : 'saga'
                }
            classifier = LogisticRegression(**params)
        
        elif classifier_name =="RF":
            params = {
                'n_estimators': trial.suggest_int("rf_n_estimators", 100, 1000),
                'criterion': trial.suggest_categorical("rf_criterion", ['gini', 'entropy']),
                'max_depth': trial.suggest_int("rf_max_depth", 1, 4),
                'min_samples_split': trial.suggest_float("rf_min_samples_split", 0.01, 1),
                }
            classifier = RandomForestClassifier(**params)

        elif classifier_name =="LGBM":
            params = {
                'n_estimators': trial.suggest_int("lgbm_n_estimators", 20, 10000),
                'num_leaves': trial.suggest_int("lgbm_num_leaves", 10, 1000),
                'max_depth': trial.suggest_int("lgbm_max_depth", 2, 20),
                'min_child_samples': trial.suggest_int("lgbm_min_child_samples", 5, 300),
                'learning_rate': trial.suggest_float('lgbm_learning_rate', 1e-5, 1e-2),
                'boosting_type': trial.suggest_categorical("lgbm_boosting_type", ['goss', 'gbdt'])
                }
            classifier = LGBMClassifier(**params)  
        
        elif classifier_name =="GBM":
            params = {
                'n_estimators': trial.suggest_int("gbm_n_estimators", 100, 1000), 
                'criterion': trial.suggest_categorical("gbm_criterion", ['squared_error', 'friedman_mse']),
                'max_depth': trial.suggest_int("gbm_max_depth", 1, 4),
                'min_samples_split': trial.suggest_int("gbm_min_samples_split", 5, 300),
                }
            classifier = GradientBoostingClassifier(**params)            
        
        else:
            None
        

        pipeline = Pipeline(
            steps=[
                ("preprocessor", preprocessor),
                ("classifier", classifier),
            ]
        )
                
        scores = cross_validate(
                pipeline, 
                X_train,
                y_train, 
                cv=5,
                scoring = metric_scorer_decorated,
                return_train_score=True)

        fair_metric = scores['test_fairness'].mean()
        model_metric = scores['test_model'].mean()

        return fair_metric, model_metric
    
    study = optuna.create_study(
        directions = directions, 
        pruner = optuna.pruners.SuccessiveHalvingPruner(), 
        sampler = optuna.samplers.TPESampler() 
        )
    
    study.optimize(objective, n_trials=n_trials, n_jobs=1)
    
    print("Number of finished trials: ", len(study.trials))
    results.append(study)
    with open(file_name, 'wb') as file:
        dill.dump(results, file)
        print(f'Object successfully saved to "{file_name}"')

[I 2023-07-02 21:02:31,823] A new study created in memory with name: no-name-b0942f29-73d5-4336-afeb-436afba302b7


0


[I 2023-07-02 21:03:21,007] Trial 0 finished with values: [0.04712843946660452, 0.6791184358153408] and parameters: {'classifier': 'GBM', 'gbm_n_estimators': 529, 'gbm_criterion': 'friedman_mse', 'gbm_max_depth': 4, 'gbm_min_samples_split': 208}. 
/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarni

Number of finished trials:  100
Object successfully saved to "recall-ppv-models-motpe-succesivehalving-parallel-100trials-1sim.pkl"
1


/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ooti/anaconda3/envs/fairenv/lib/python3.10/site-pack

Number of finished trials:  100
Object successfully saved to "recall-ppv-models-motpe-succesivehalving-parallel-100trials-2sim.pkl"
2


[I 2023-07-03 00:26:08,665] Trial 0 finished with values: [0.04689258258520486, 0.6748494923788413] and parameters: {'classifier': 'GBM', 'gbm_n_estimators': 966, 'gbm_criterion': 'friedman_mse', 'gbm_max_depth': 3, 'gbm_min_samples_split': 230}. 
[I 2023-07-03 00:26:59,494] Trial 1 finished with values: [0.047173888683212414, 0.6692989537439911] and parameters: {'classifier': 'GBM', 'gbm_n_estimators': 708, 'gbm_criterion': 'friedman_mse', 'gbm_max_depth': 3, 'gbm_min_samples_split': 212}. 
[I 2023-07-03 00:28:23,711] Trial 2 finished with values: [0.02855035862423596, 0.6921455545521715] and parameters: {'classifier': 'LGBM', 'lgbm_n_estimators': 8644, 'lgbm_num_leaves': 649, 'lgbm_max_depth': 8, 'lgbm_min_child_samples': 82, 'lgbm_learning_rate': 0.007768887892223484, 'lgbm_boosting_type': 'goss'}. 
[I 2023-07-03 00:28:33,714] Trial 3 finished with values: [0.018865726715755794, 0.1426121281777632] and parameters: {'classifier': 'RF', 'rf_n_estimators': 502, 'rf_criterion': 'gini', 